In [1]:
import gradio as gr

In [2]:
import pandas as pd
import json
import numpy as np
import spacy
import string
import nltk
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings("ignore")
from spacy import tokenizer
from spacy.lang.en import English
from nltk.tokenize import sent_tokenize
import re
from bs4 import BeautifulSoup
import unicodedata
import pickle

### It's necessary to clean the input of the post so we report again the preprocessing

In [3]:
contractions_dict = {"ain't": "are not", "'s":" ", " s ":" is", "aren't": "are not", "Aren't": "are not", "Can't": "can not", "can't": "can not", "can't": "cannot", "can't've": "cannot have", "'cause": "because", "could've": "could have", "couldn't": "could not", "couldn't've": "could not have", "didn't": "did not", "doesn't": "does not", "don't": "do not", "don t": "do not", "dont": "do not", "dunno": "do not know", "hadn't": "had not", "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not", "he'd": "he would", "he'd've": "he would have", "he'll": "he will", " hes ": " he is ", "he'll've": "he will have", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "i'll": "i will", "'ll":" will", "I'll've": "I will have", "I’m": "I am", "i'm": "i am", "'m": " am", " im ": " i am ", "I've": "I have", "i've": "i have", "I've": "I have", "havent": "have not", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll’ve": "it will have", "let's": "let us", "lets": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have", "mightn't": "might not", "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't’ve": "need not have", "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "She'll": "she will", "she'll": "she will", "she'll've": "she will have", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have", "that'd": "that would", "that'd've": "that would have", "there'd": "there would", "there'd've": "there would have", "they'd": "they would", "they'd've": "they would have","they'll": "they will", "wont": "will not",
  "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not","what'll": "what will", "what'll've": "what will have", "what're": "what are", "what've": "what have", "when've": "when have", "where’d": "where did", "where've": "where have",
  "who'll": "who will", "who'll've": "who will have", "who've": "who have", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y’all": "you all", "y'all'd": "you all would", "y'all'd've": "you all would have", "y'all're": "you all are", "y'all've": "you all have", "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "You'll": "you will", "you'll've": "you will have", "You're": "you are", "you're": "you are", "you've": "you have", "wanna": "want to", " u ": " you ", " r ": " are ", "gawd": "god", "urlLink": "", "luv": "love", "Luv": "Love", "wuts": "what is", "wasnt": "was not", "Wasnt": "was not", " bday ": "birthday", " gotta ": "got to", "gonna": "going to",
  "nvr": "never", " cuz ": " because ", " cos ": " because "," cant ": " can not ", "yr": " years old", "i\'d": "i\ would", "they\'d": "they\ would", "we\'re": "we\ are", "shouldn\'t": "should\ not", "Don\'t": "Do\ not", "won\'t": "will\ not", "haven\'t": "have\ not", "you\'re": "you\ are", "they\'re": "they\ are", "Didn\'t": "Did\ not", "Hasn\'t": "Has\ not", "I\'d": "I\ would", "you\'ve": "you\ have", "peeps": "people", "it's": "it is", "kinda": "kind of", "buyin": "buying", "Its": "It is", "bout": "about", " ppl ": " people ", " n ": " and ", "enuf": "enough", "btw": "by the way", "BTW": "BY THE WAY", 'b/c': "because", " aabout": "about", " aaabout": "about", "aaaabout": "about"}

In [4]:
contractions_re = re.compile('(%s)'%'|'.join(contractions_dict.keys()))
def expand_contractions(s, contractions_dict=contractions_dict):
  def replace(match):
    return contractions_dict[match.group(0)]
  return contractions_re.sub(replace, s)
def lowering(text):
    return text.lower()
def remove_url(text):
    return re.sub(r'https?://\S+|www\.\S+', '', text)
def remove_html_tags_func(text):
    return BeautifulSoup(text, 'html.parser').get_text()
def remove_accents(text):
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
def remove_punctuation(text):
    # return re.sub(r'[^a-zA-Z0-9]', ' ', text) # --> if you allow for numbers
    return re.sub(r'[^a-zA-Z]', ' ', text) # --> if you do not allow for numbers
def remove_extra_spaces(text):
    return re.sub(r'^\s*|\s\s*', ' ', text).strip()

en = spacy.load('en_core_web_sm')
sw_spacy = en.Defaults.stop_words

def delete_stopwords(text):
    words = [word for word in text.split() if word.lower() not in sw_spacy]
    new_text = " ".join(words)
    return new_text

In [5]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2') 

In [6]:
#Funzione che faccia il preprocessing
def preprocessing(post):
    post = expand_contractions(post)
    post = lowering(post)
    post = remove_url(post)
    post = remove_html_tags_func(post)
    post  = remove_accents(post)
    post = remove_punctuation(post)
    post = remove_extra_spaces(post)
    post = delete_stopwords(post)
    return post

# NAIVE BAYES

In [7]:
df = pd.read_pickle("train_clean_2.pkl")
df['age_class'] = pd.cut(
        df["age"],
        bins=[12, 18, 28, 50],
        labels=[0, 1, 2]
    ).astype("int32")
X_train = df.post
y_train = df.age_class
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import balanced_accuracy_score
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)

In [11]:
def profAiling_age(post):
    post = [preprocessing(post)]
    predicted = nb.predict_proba(post).tolist()[0]
    class_names = ["Teens", "Twenties", "Tirthies"]
    return {class_names[i]: predicted[i] for i in range(3)}

# Logistic Regression

In [12]:
X_train = df.post
y_train = df.gender
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
               ])
logreg.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(C=100000.0, n_jobs=1))])

In [14]:
#la funzione dentro Gradio
def profAiling(post):
    post = [preprocessing(post)]
    predicted = logreg.predict_proba(post).tolist()[0]
    class_names = ["Female", "Male"]
    return {class_names[i]: predicted[i] for i in range(2)}

In [15]:
profAiling_age("LOve LOL shopping mum")

{'Teens': 0.7343224758004311,
 'Twenties': 0.22966412946430942,
 'Tirthies': 0.036013394735259496}

In [16]:
profAiling("War American President")

{'Female': 0.36295984895718403, 'Male': 0.637040151042816}

In [17]:
app_1 =  gr.Interface(fn = profAiling, 
                    inputs="text", 
                    outputs=gr.outputs.Label(),
                    description="Welcome to profAIling app!\n\nSubmit your post and we will guess if you are a male or a female",
                    examples = ["Rome is really hot in summer... Let's go shopping", "I really love the statistical learning project!", "I want to go on my vacation after a stressing year",])

In [18]:
app_2 = gr.Interface(fn = profAiling_age, 
                    inputs="text", 
                    outputs=gr.outputs.Label(),
                    description="Welcome to profAIling app!\n\nSubmit your post and we will guess your age",
                    examples = ["Rome is really hot in summer... Let's go shopping", "I really love the statistical learning project!", "I want to go on my vacation after a stressing year",])

In [22]:
demo = gr.TabbedInterface([app_1, app_2], ["Gender", "Age"], title = "ProfAIling")

In [28]:
demo.launch(share=True)



Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Running on public URL: https://51637979c533a1bcba.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
